In [1]:
import torch
import os
from tqdm import tqdm
from PIL import Image

In [6]:
!wget https://www.di.ens.fr/willow/teaching/recvis18orig/assignment3/bird_dataset.zip
!unzip -q bird_dataset.zip 
!rm bird_dataset.zip

--2021-11-14 00:54:12--  https://www.di.ens.fr/willow/teaching/recvis18orig/assignment3/bird_dataset.zip
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘bird_dataset.zip’

bird_dataset.zip        [         <=>        ] 183,48M  2,39MB/s    in 32s     

2021-11-14 00:54:44 (5,66 MB/s) - ‘bird_dataset.zip’ saved [192388716]



In [1]:
def get_best_bird(results):
    # Return the best bird found in an image
    bounding_boxes = results.pandas().xyxy[0]
    best_confidence = 0.
    arg_best = None
    for index, row in bounding_boxes.iterrows():
        if (row['name'] == 'bird') and (row['confidence'] > best_confidence):
            best_confidence = row['confidence']
            arg_best = row.tolist()
    return arg_best
            
def crop_birds(folder):
    # Crop bird from test_images/ val_images/ train_images/
    model = torch.hub.load('ultralytics/yolov5', 'yolov5x6') 
    for root, dirs, files in os.walk(folder):
        for img in tqdm(files):
            img_path = os.path.join(root,img)
            results = model(img_path)
            best_bird_box = get_best_bird(results)
            if best_bird_box is None:
                print(f"Couldn't find bird in {root}/{img}")
            else:
                im = Image.open(img_path)
                left, top, right, bottom = best_bird_box[:4]
                cropped = im.crop((left, top, right, bottom))
                cropped.save(img_path)
                
def crop_single_image(img_path):
    model = torch.hub.load('ultralytics/yolov5', 'yolov5x') 
    results = model(img_path)
    best_bird_box = get_best_bird(results)
    if best_bird_box is None:
        print(f"Couldn't find bird in {img_path}")
    else:
        im = Image.open(img_path)
        left, top, right, bottom = best_bird_box[:4]
        cropped = im.crop((left, top, right, bottom))
        cropped.save(img_path)

In [3]:
crop_birds('bird_dataset/val_images')

Using cache found in /home/loic/.cache/torch/hub/ultralytics_yolov5_master
/home/loic/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0
YOLOv5 🚀 2021-11-13 torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 
0it [00:00, ?it/s]
100%|██████████| 8/8 [00:11<00:00,  1.43s/it]


In [4]:
crop_birds('bird_dataset/train_images')

Using cache found in /home/loic/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-11-13 torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 
0it [00:00, ?it/s]
100%|██████████| 52/52 [01:13<00:00,  1.41s/it]


In [5]:
crop_birds('bird_dataset/test_images')

Using cache found in /home/loic/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-11-13 torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 
0it [00:00, ?it/s]
  0%|          | 2/517 [00:02<08:59,  1.05s/it]

Couldn't find bird in bird_dataset/test_images/mistery_category/0247efd7b9d47d036bb4390202a13e69.jpg


 18%|█▊        | 94/517 [02:01<09:06,  1.29s/it]

Couldn't find bird in bird_dataset/test_images/mistery_category/8ede0bc5a4976385dcfe6e38feaf90c2.jpg


100%|██████████| 517/517 [10:08<00:00,  1.18s/it]


In [14]:
"""
In two images birds are not found, so I test other models
For the image test_images/mistery_category/8ede0bc5a4976385dcfe6e38feaf90c2.jpg nothing worked so I
eventually cropped it myself
"""
crop_single_image('bird_dataset/test_images/mistery_category/0247efd7b9d47d036bb4390202a13e69.jpg')

Using cache found in /home/loic/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-11-13 torch 1.9.0+cu102 CPU



  0%|          | 0.00/89.2M [00:00<?, ?B/s]

Fusing layers... 


Model Summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


In [2]:
!zip -r bird_dataset_cropped_yolov5.zip bird_dataset/*

  adding: bird_dataset/test_images/ (stored 0%)
  adding: bird_dataset/test_images/mistery_category/ (stored 0%)
  adding: bird_dataset/test_images/mistery_category/ad08ba64f41b7c50feb7701489a25fc9.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/0247efd7b9d47d036bb4390202a13e69.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/57e0aecd931044c03adde0ff6e944b4c.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/3e127d066d153ab419d86c593f77d629.jpg (deflated 0%)
  adding: bird_dataset/test_images/mistery_category/964581a80041d2f57c4dd40a82ad94dd.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/f540d62813721058081c65c41a9cb0dc.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/99a8b2b93391f72f4437c4bf4465501e.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/1d46afe83c6d66741ec950843307f6e6.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/8992398fc52

  adding: bird_dataset/test_images/mistery_category/5db472b43f5627d4968c40a33278734c.jpg (deflated 7%)
  adding: bird_dataset/test_images/mistery_category/584fb865fbd41948c0fd3750923b29c3.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/f067618d20c64929543b7a7570f8e629.jpg (deflated 3%)
  adding: bird_dataset/test_images/mistery_category/fb97b3a7197b6f5f856782d39fa14f49.jpg (deflated 15%)
  adding: bird_dataset/test_images/mistery_category/ac9734b702267d0b1016ae7d41d62928.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/6ebdc52674fa9872a716060f4353b6fb.jpg (deflated 4%)
  adding: bird_dataset/test_images/mistery_category/3cff6b88d93b15c1118ddda2892339c6.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/a3a5f7837055db69f10649dd7cf6a92c.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/a3d9980ac0e9517507ffedb37779895a.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/7c18125deb9cddeba47c

  adding: bird_dataset/test_images/mistery_category/be7ed1c27780067717d28964044acf51.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/b4b2bb2d96768ef960b53ffc3b3e7d1b.jpg (deflated 3%)
  adding: bird_dataset/test_images/mistery_category/f3ffaa63cce4b75f414d93f65bd45743.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/b9e87230db892717fdc5a0513e312ccb.jpg (deflated 5%)
  adding: bird_dataset/test_images/mistery_category/14dfa660b37a9ee2712c38f9002d8980.jpg (deflated 4%)
  adding: bird_dataset/test_images/mistery_category/5c1d17bfbf543407b28d6a2f7932410c.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/b193593ec3ccc440508145cae0ef086e.jpg (deflated 5%)
  adding: bird_dataset/test_images/mistery_category/20e812366b1c572222f1fa1e3d54dc54.jpg (deflated 4%)
  adding: bird_dataset/test_images/mistery_category/8a94c643402bd103bb29e4ad2c7f4e78.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/465ce159f0171470a2826

  adding: bird_dataset/test_images/mistery_category/002f61512a368e4c1434eedacf609957.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/0ecc886c88bf3845dce95f2281153b22.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/226cd33fa66e7d7ed28a357e1a3ae3d4.jpg (deflated 11%)
  adding: bird_dataset/test_images/mistery_category/f6a544074a9b83ccd98bf65a2c3eb57c.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/532bdf3560359a3be1988d9e68bd39b1.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/6ff9b514f288e9907b136ba36854a1eb.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/501505f4ad95c828a271445f5073a91d.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/371e69a08d83234cc16e353734885d26.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/83a5afccfd69121b1fa338d5f37379a1.jpg (deflated 3%)
  adding: bird_dataset/test_images/mistery_category/5414fa19cb93ac5c913b

  adding: bird_dataset/test_images/mistery_category/0910f1d5badf42de1aa9bae05037412d.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/d08b43687503170c36402386df6a490f.jpg (deflated 5%)
  adding: bird_dataset/test_images/mistery_category/724120169552f831a8e17ea8ce8447a4.jpg (deflated 0%)
  adding: bird_dataset/test_images/mistery_category/05c985774004cd0e789e33f2a00ca26a.jpg (deflated 8%)
  adding: bird_dataset/test_images/mistery_category/6e3c2c583774606210735e93fda8a521.jpg (deflated 3%)
  adding: bird_dataset/test_images/mistery_category/c8ec34cf7ee54d51bb632a53d89f5c56.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/4e7e72cbf3b0c9259dea6fc6601786b6.jpg (deflated 11%)
  adding: bird_dataset/test_images/mistery_category/4ac18803f100031f5f87c1b78d62049c.jpg (deflated 2%)
  adding: bird_dataset/test_images/mistery_category/97a9bdf7f7c32a8b07d4dfbd288bc2d0.jpg (deflated 1%)
  adding: bird_dataset/test_images/mistery_category/59e5a4b2bde751759781

...

  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0102_6590.jpg (deflated 1%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0081_6967.jpg (deflated 1%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0055_6923.jpg (deflated 6%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0016_6684.jpg (deflated 1%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0057_6935.jpg (deflated 3%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0026_6768.jpg (deflated 2%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0073_6744.jpg (deflated 3%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0091_6695.jpg (deflated 1%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0086_6658.jpg (deflated 2%)
  adding: bird_dataset/train_images/011.Rusty_Blackbird/Rusty_Blackbird_0033_6879.jpg (deflated 1%)


  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0002_21819.jpg (deflated 2%)
  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0073_21932.jpg (deflated 2%)
  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0022_21944.jpg (deflated 1%)
  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0089_21804.jpg (deflated 2%)
  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0012_21961.jpg (deflated 1%)
  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0008_21856.jpg (deflated 2%)
  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0100_21913.jpg (deflated 2%)
  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0033_21873.jpg (deflated 3%)
  adding: bird_dataset/train_images/020.Yellow_breasted_Chat/Yellow_Breasted_Chat_0084_22082.jpg (deflated 2%)
 

  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0029_797300.jpg (deflated 1%)
  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0019_27192.jpg (deflated 1%)
  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0028_27114.jpg (deflated 1%)
  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0066_797298.jpg (deflated 1%)
  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0041_27105.jpg (deflated 1%)
  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0042_27143.jpg (deflated 2%)
  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0040_26985.jpg (deflated 1%)
  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0038_797309.jpg (deflated 1%)
  adding: bird_dataset/train_images/034.Gray_crowned_Rosy_Fin

  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0138_20945.jpg (deflated 1%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0127_20034.jpg (deflated 3%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0067_21043.jpg (deflated 1%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0141_21174.jpg (deflated 1%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0100_20674.jpg (deflated 1%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0129_20987.jpg (deflated 1%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0130_20328.jpg (deflated 1%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0105_20864.jpg (deflated 2%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0053_20694.jpg (deflated 1%)
  adding: bird_dataset/train_images/019.Gray_Catbird/Gray_Catbird_0013_20562.jpg (deflated 1%)
  adding: bird_dataset/train_images/019.Gray_Catbi

  adding: bird_dataset/val_images/023.Brandt_Cormorant/Brandt_Cormorant_0009_22890.jpg (deflated 2%)
  adding: bird_dataset/val_images/023.Brandt_Cormorant/Brandt_Cormorant_0083_23156.jpg (deflated 2%)
  adding: bird_dataset/val_images/023.Brandt_Cormorant/Brandt_Cormorant_0070_23137.jpg (deflated 3%)
  adding: bird_dataset/val_images/023.Brandt_Cormorant/Brandt_Cormorant_0039_22945.jpg (deflated 1%)
  adding: bird_dataset/val_images/009.Brewer_Blackbird/ (stored 0%)
  adding: bird_dataset/val_images/009.Brewer_Blackbird/Brewer_Blackbird_0127_2235.jpg (deflated 1%)
  adding: bird_dataset/val_images/009.Brewer_Blackbird/Brewer_Blackbird_0104_2230.jpg (deflated 3%)
  adding: bird_dataset/val_images/009.Brewer_Blackbird/Brewer_Blackbird_0015_2286.jpg (deflated 2%)
  adding: bird_dataset/val_images/009.Brewer_Blackbird/Brewer_Blackbird_0009_2616.jpg (deflated 1%)
  adding: bird_dataset/val_images/030.Fish_Crow/ (stored 0%)
  adding: bird_dataset/val_images/030.Fish_Crow/Fish_Crow_0069_2600